## IMPORTS

In [5]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models

## CONFIGURATION

In [6]:
DATASET_PATH = r"C:\PROJECTS\ai_ml\Memowee\MEMOWEE_DATASET\train"  # your known users folder
MODEL_PATH = "trained_models_and_embeddings_files\MEMOWEE_EMBEDDING_MODEL.h5"  # trained embedding model
OUTPUT_EMBEDDINGS = "MEMOWEE_DATASET_EMBEDDINGS.npz"
IMG_SIZE = (128, 128)

## L2 NORMALIZATION LAYER

In [7]:
class L2Normalization(layers.Layer):
    def call(self, inputs):
        return tf.math.l2_normalize(inputs, axis=1)

    def get_config(self):
        return super(L2Normalization, self).get_config()

## LOAD TRAINED EMBEDDING MODEL

In [8]:
embedding_model = tf.keras.models.load_model(
    MODEL_PATH,
    compile=False,
    custom_objects={'L2Normalization': L2Normalization}
)
print("Embedding model loaded successfully!")


Embedding model loaded successfully!


## HELPER TO PREPROCESS IMAGES

In [9]:
def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, IMG_SIZE)
    img = img / 255.0
    img = np.expand_dims(img, axis=0)  # batch dimension
    return img

## LOOP THROUGH DATASET AND EXTRACT EMBEDDINGS

In [10]:
embeddings_dict = {}

class_folders = sorted(os.listdir(DATASET_PATH))
for cls_name in class_folders:
    cls_path = os.path.join(DATASET_PATH, cls_name)
    if not os.path.isdir(cls_path):
        continue
    
    embeddings_dict[cls_name] = []
    for img_file in os.listdir(cls_path):
        if img_file.lower().endswith((".jpg", ".jpeg", ".png")):
            img_path = os.path.join(cls_path, img_file)
            img_array = preprocess_image(img_path)
            
            emb = embedding_model.predict(img_array)
            embeddings_dict[cls_name].append(emb.squeeze())
    
    print(f"[INFO] Extracted {len(embeddings_dict[cls_name])} embeddings for user '{cls_name}'")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━

## SAVE EMBEDDINGS TO NPZ

In [11]:
np.savez_compressed(OUTPUT_EMBEDDINGS, **embeddings_dict)
print(f"\n[INFO] All embeddings saved to {OUTPUT_EMBEDDINGS}")


[INFO] All embeddings saved to MEMOWEE_DATASET_EMBEDDINGS.npz
